In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ShuffleSplit, KFold
from functools import partial
from skopt import space, gp_minimize
from sklearn.metrics import mean_squared_error as mse

def optimize(params, param_names, x, y):
    params = dict(zip(param_names, params))
    model = RandomForestRegressor(**params)
    
    mse_scores = []
    
    kfold = KFold(n_splits=10, shuffle=True)
    for train_index, test_index in kfold.split(x):
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(x_train, y_train)
        predict = model.predict(x_test)
        score = mse(y_test, predict)
        mse_scores.append(score)
    return np.mean(mse_scores)

param_space = [space.Integer(1, 25, name='max_depth'), 
               space.Integer(20, 200, name='n_estimators'), 
               space.Real(0.1, 0.99, prior='uniform', name='max_features'), 
               space.Real(0.1, 0.99, prior='uniform', name='max_samples')]

param_names = ['max_depth', 'n_estimators', 
               'max_features', 'max_samples',
               'min_samples_split', 'min_samples_leaf']

optimization_func = partial(optimize, 
                            param_names=param_names,
                            x=x,
                            y=scaled_scores)

result = gp_minimize(optimization_func, param_space, n_calls=30, verbose=10)
print(dict(zip(param_names, result.x)))

ModuleNotFoundError: No module named 'skopt'